## Basic properties of NestedTensor

This notebook illustries some of the basic properties of NestedTensor such as dim, size and nested_size.

In [1]:
from nestedtensor import torch
def print_eval(s):
    print(("\033[1;31m$ " + s + ":\033[0m").ljust(30) + "\n{}\n".format(str(eval(s))))

Imagine the following is a collection of Grey-scale images. The NestedTensor represents a list with two entries. The first entry of that list is a list of two images, the second entry of that list is a list with one image. Maybe these images represent faces and are grouped by the corresponding individual.

If you find this example to be too unrealistic, imagine that these Tensors represents sentences group in paragraph and are of int64 instead of float. Or imagine that they represent multi-channel audio files grouped by speaker identitiy.

In [2]:
nt = torch.nested_tensor(
    [
        [
            torch.rand(2, 3),
            torch.rand(4, 5)
        ],
        [
            torch.rand(1, 2)
        ]
    ])
print_eval("nt")

$ nt:              
nested_tensor([
	[
		tensor([[0.7173, 0.5520, 0.9007],
		        [0.4070, 0.6530, 0.8580]]),
		tensor([[0.6783, 0.9212, 0.8009, 0.4163, 0.9832],
		        [0.3170, 0.8630, 0.1032, 0.8065, 0.2567],
		        [0.0512, 0.0900, 0.8160, 0.7853, 0.1891],
		        [0.5247, 0.8860, 0.6753, 0.0171, 0.6665]])
	],
	[
		tensor([[0.9114, 0.7392]])
	]
])



In [3]:
# Every non-empty NestedTensor is of at least dimension one, because it must represent at least a list.
# For each level lists with list entries added we increase the nested dimension by one. That means
# this NestedTensor is of dimension two.
print_eval("nt.nested_dim()")

# The tensor dimension is two, because the Tensor constiuents are of dimension two.
print_eval("nt.tensor_dim()")

# The dimension is four, because it is the sum of the nested and tensor dimension.
print_eval("nt.dim()")



$ nt.nested_dim(): 
2

$ nt.tensor_dim(): 
2

$ nt.dim():        
4



In [4]:
# The data type, layout and device of a NestedTensor as unsurprisingly that of the Tensor constiuent.
# Just as with torch.tensor these properties must align during construction.
print_eval("nt.dtype")
print_eval("nt.layout")
print_eval("nt.device")

$ nt.dtype:        
torch.float32

$ nt.layout:       
torch.strided

$ nt.device:       
cpu



### torch.nested_tensor_from_tensor_mask, torch.NestedTensor.to_tensor_mask and more
To put NestedTensors in context of current approaches of dealing with variably sized datapoints, such as padding and masking, we will introduce construction and conversion to tensors with masks and tensors with speical non-data identifying values.

In [5]:
tensor = torch.tensor(
        [[[0.8413, 0.7325, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.6334, 0.5473, 0.3273, 0.0564],
         [0.3023, 0.6826, 0.3519, 0.1804],
         [0.8431, 0.1645, 0.1821, 0.9185]]])
mask = torch.tensor(
        [[[ True,  True, False, False],
         [False, False, False, False],
         [False, False, False, False]],

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]])
print_eval("tensor")
print_eval("mask")
nt2 = torch.nested_tensor_from_tensor_mask(tensor, mask)
print_eval("torch.nested_tensor_from_tensor_mask(tensor, mask)")
print_eval("torch.nested_tensor_from_padded_tensor(tensor, padding=0)")

$ tensor:          
tensor([[[0.8413, 0.7325, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.6334, 0.5473, 0.3273, 0.0564],
         [0.3023, 0.6826, 0.3519, 0.1804],
         [0.8431, 0.1645, 0.1821, 0.9185]]])

$ mask:            
tensor([[[ True,  True, False, False],
         [False, False, False, False],
         [False, False, False, False]],

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]])

$ torch.nested_tensor_from_tensor_mask(tensor, mask):
nested_tensor([
	tensor([[0.8413, 0.7325]]),
	tensor([[0.6334, 0.5473, 0.3273, 0.0564],
	        [0.3023, 0.6826, 0.3519, 0.1804],
	        [0.8431, 0.1645, 0.1821, 0.9185]])
])

$ torch.nested_tensor_from_padded_tensor(tensor, padding=0):
nested_tensor([
	tensor([[0.8413, 0.7325]]),
	tensor([[0.6334, 0.5473, 0.3273, 0.0564],
	        [0.3023, 0.6826, 0.3519, 0.1804],
	        [0.8431, 0.1645, 0.1821,

In [6]:
print_eval("nt2.to_tensor_mask()")
print_eval("nt2.to_padded_tensor(padding=-10)")

$ nt2.to_tensor_mask():
(tensor([[[0.8413, 0.7325, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.6334, 0.5473, 0.3273, 0.0564],
         [0.3023, 0.6826, 0.3519, 0.1804],
         [0.8431, 0.1645, 0.1821, 0.9185]]]), tensor([[[ True,  True, False, False],
         [False, False, False, False],
         [False, False, False, False]],

        [[ True,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]]))

$ nt2.to_padded_tensor(padding=-10):
tensor([[[  0.8413,   0.7325, -10.0000, -10.0000],
         [-10.0000, -10.0000, -10.0000, -10.0000],
         [-10.0000, -10.0000, -10.0000, -10.0000]],

        [[  0.6334,   0.5473,   0.3273,   0.0564],
         [  0.3023,   0.6826,   0.3519,   0.1804],
         [  0.8431,   0.1645,   0.1821,   0.9185]]])



**nested_size, size and len()** should be part of the bread and butter of a NestedTensor user.

Therefore it is important to understand these concepts well.

NestedTensor.nested_size is defined as the result of recusrively mapping ```lambda x: x.size()``` onto a NestedTensor's tensor constiuents. Or more loosely defined, it is the result of replacing the Tensor constiuents by their size.

NestedTensor.nested_size optionally also accepts a dim argument. This will return a slice across the given dimension. This might be easiest explain via below example.

nt.nested_size(0) returns the length of nt or the number of entries in the list it represents. This is very similar to ```list.__len__```.

nt.nested_size(1) returns the length of the entries of the outer list.

nt.nested_size(2) returns the first entry of each Tensor constiuent's size. 

nt.nested_size(3) returns the second entry of each Tensor constiuent's size.

We will soon define .size and unbind which will make the definition of this even clearer. We will also show some examples that justify these methods.


In [7]:
print_eval("nt")
print_eval("nt.nested_size()")
print_eval("len(nt)")
print_eval("nt.nested_size(0)")
print_eval("nt.nested_size(1)")
print_eval("nt.nested_size(2)")
print_eval("nt.nested_size(3)")

$ nt:              
nested_tensor([
	[
		tensor([[0.7173, 0.5520, 0.9007],
		        [0.4070, 0.6530, 0.8580]]),
		tensor([[0.6783, 0.9212, 0.8009, 0.4163, 0.9832],
		        [0.3170, 0.8630, 0.1032, 0.8065, 0.2567],
		        [0.0512, 0.0900, 0.8160, 0.7853, 0.1891],
		        [0.5247, 0.8860, 0.6753, 0.0171, 0.6665]])
	],
	[
		tensor([[0.9114, 0.7392]])
	]
])

$ nt.nested_size():
torch.NestedSize((
	(
		torch.Size([2, 3]),
		torch.Size([4, 5])
	),
	(
		torch.Size([1, 2])
	)
))

$ len(nt):         
2

$ nt.nested_size(0):
2

$ nt.nested_size(1):
(2, 1)

$ nt.nested_size(2):
((2, 4), (1,))

$ nt.nested_size(3):
((3, 5), (2,))



**NestedTensor.size** is a function that returns a tuple of the format
(n_1, n_2, ..., n_nested_dim, t_1, t_2, ..., t_tensor_dim). The sizes lead by n_ are defined 
to be the nested sizes each at a nested dimension, the sizes lead by t_ are defined to be the 
tensor sizes each at a tensor dimension. They are a reduced version of nested_size and 
aim to represent the size across a slice of nested_size.

size(i) is of value k if all numerical entries of nested_size(dim) are of value k, otherwise it is None.
size() is a tuple with entries size(i)
In this case most size(i) will be None, except for the first. We will later see examples of NestedTensors where this is not the case

In [8]:
print_eval("nt.size()")

$ nt.size():       
(2, None, None, None)



**unbind** is a fundamental building block of NestedTensors. Applying unbind to a NestedTensor will return the constiuents of the list it represents. More importantly, it returns a few of these elements. It does not take a dim argument, for now, in comparison to torch.Tensor.unbind.

In [9]:
entries = nt.unbind()
print(entries[0])
print("")
print(entries[1])

nested_tensor([
	tensor([[0.7173, 0.5520, 0.9007],
	        [0.4070, 0.6530, 0.8580]]),
	tensor([[0.6783, 0.9212, 0.8009, 0.4163, 0.9832],
	        [0.3170, 0.8630, 0.1032, 0.8065, 0.2567],
	        [0.0512, 0.0900, 0.8160, 0.7853, 0.1891],
	        [0.5247, 0.8860, 0.6753, 0.0171, 0.6665]])
])

nested_tensor([
	tensor([[0.9114, 0.7392]])
])


In [10]:
# Edit the first entry of the first list in-place. You can see that the memory is shared between these constructs.
entries[0].unbind()[0].cos_()
print(nt)

nested_tensor([
	[
		tensor([[0.7536, 0.8515, 0.6210],
		        [0.9183, 0.7943, 0.6540]]),
		tensor([[0.6783, 0.9212, 0.8009, 0.4163, 0.9832],
		        [0.3170, 0.8630, 0.1032, 0.8065, 0.2567],
		        [0.0512, 0.0900, 0.8160, 0.7853, 0.1891],
		        [0.5247, 0.8860, 0.6753, 0.0171, 0.6665]])
	],
	[
		tensor([[0.9114, 0.7392]])
	]
])
